# Ternarized Convolutional Neural Networks

This tutorial demonstrates training a simple Ternarized Convolutional Neural Network to classify MNIST digits. It uses the same model architecture as the Binarized example 

### Import TensorFlow and Larq

In [1]:
import tensorflow as tf
import larq as lq

### Download and prepare the MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1., test_images / 127.5 - 1.

### Create the model

In [3]:
kwargs = dict(input_quantizer=lq.quantizers.SteTern(threshold_value=0.05), kernel_quantizer=lq.quantizers.SteTern(threshold_value=0.05), kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

model.add(lq.layers.QuantConv2D(32, (3, 3), kernel_quantizer=lq.quantizers.SteTern(threshold_value=0.05), kernel_constraint="weight_clip", use_bias=False, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

Here's the complete architecture of our model.

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quant_conv2d (QuantConv2D)   (None, 26, 26, 32)        288       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 32)        96        
_________________________________________________________________
quant_conv2d_1 (QuantConv2D) (None, 11, 11, 64)        18432     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 64)          192       
_________________________________________________________________
quant_conv2d_2 (QuantConv2D) (None, 3, 3, 64)          36864     
__________

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=50, epochs=10)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/10
60000/60000 [==============================] - 8s 137us/step - loss: 0.5812 - acc: 0.9297
Epoch 2/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4554 - acc: 0.9675
Epoch 3/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.4343 - acc: 0.9746
Epoch 4/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.4207 - acc: 0.9783
Epoch 5/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.4187 - acc: 0.9795
Epoch 6/10
60000/60000 [==============================] - 6s 105us/step - loss: 0.4179 - acc: 0.9811
Epoch 7/10
60000/60000 [==============================] - 6s 104us/step - loss: 0.4130 - acc: 0.9822
Epoch 8/10
60000/60000 [==============================] - 6s 104us/step - loss: 0.4117 - acc: 0.9827
Epoch 9/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.4094 - acc: 0.9837
Epoch 10/10
10000/10000 [==============================] - 1s 53us/step


### Evaluate the model

In [6]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 98.90 %


As you can see, our simple binarized CNN has achieved a test accuracy of over 97.5 %. Not bad for a few lines of code!